**Setup**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

data_new = pd.read_csv('/content/gdrive/My Drive/pd_diagnosis/Zeno Paper/scaled_df.csv')
data_new.drop("Unnamed: 0", axis=1, inplace=True)
diagnosis_zero_rows = data_new[data_new['diagnosis'] == 0]
data_new = data_new.drop(diagnosis_zero_rows.index[:40])
data_new.head()

,diagnosis,age,tap_iti_consistency,left_taps,total_taps,right_taps,tap_accel_spectral_spread,tap_accel_spectral_centroid,tap_accel_std_z,tap_accel_mean_freq,tap_accel_std_x,gait_accel_std_z,tap_accel_rms,Spectral_Centroid_mean,tap_accel_std_y,gait_accel_rms_z
1,1,1.055015,0.095887,0.036131,0.092015,0.129903,0.259967,0.402526,0.155738,0.355077,-0.001747,0.274168,0.079586,-0.151273,-0.048371,0.259153
2,1,0.639558,0.037065,-0.372970,-0.398182,-0.376501,0.138048,-0.036333,-0.052034,-0.032050,0.025425,0.178524,-0.143964,0.315010,-0.016278,0.122602
4,1,0.778044,0.338661,-0.418425,-0.427891,-0.389486,-0.106831,-0.086215,0.149306,-0.076053,0.091212,-0.117467,0.014885,-0.195404,0.080355,-0.091860
6,1,0.824206,-0.231746,0.551296,0.634203,0.636307,0.348747,0.350325,0.230394,0.309030,0.328867,-0.099226,0.055979,0.121040,0.229858,-0.098444
8,1,0.731882,-0.515577,-0.660855,-0.635854,-0.545303,-0.418530,-0.344077,-0.231734,-0.303518,-0.155231,0.072908,-0.005120,-0.039569,-0.119340,0.059443


**Quantum SVM Preparation**

In [4]:
!pip install pennylane

Quantum Kernel

In [9]:
import pennylane as qml
from pennylane import numpy as np
import math

feature_importances = np.array([
    0.082030,  # age
    0.046870,  # tap_iti_consistency
    0.045338,  # left_taps
    0.043864,  # total_taps
    0.038588,  # right_taps
    0.029348,  # tap_accel_spectral_spread
    0.026897,  # tap_accel_spectral_centroid
    0.026357,  # tap_accel_std_z
    0.023726,  # tap_accel_mean_freq
    0.020974,  # tap_accel_std_x
    0.019336,  # gait_accel_std_z
    0.019327,  # tap_accel_rms
    0.019190,  # Spectral_Centroid_mean
    0.017527,  # tap_accel_std_y
    0.017189   # gait_accel_rms_z
])

def softmax_temperature(x, T=1.0):
    e_x = np.exp((x - np.max(x)) / T)
    return e_x / e_x.sum()

n_qubits = len(feature_importances)
dev = qml.device("default.qubit", wires=n_qubits)

def aggregate_measurements(measurements):
    return np.sum(softmax_temperature(feature_importances, T=0.01) * measurements)

@qml.qnode(dev)
def quantum_kernel(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_kernel_function(x1, x2):
    measurements = quantum_kernel(x1, x2)
    return aggregate_measurements(measurements)

def kernel_matrix(X1, X2):
    return np.array([[quantum_kernel_function(x1, x2) for x2 in X2] for x1 in X1])

Quantum kernel with detailed logs

In [10]:
import pennylane as qml
from pennylane import numpy as np
import math

feature_importances = np.array([
    0.082030,  # age
    0.046870,  # tap_iti_consistency
    0.045338,  # left_taps
    0.043864,  # total_taps
    0.038588,  # right_taps
    0.029348,  # tap_accel_spectral_spread
    0.026897,  # tap_accel_spectral_centroid
    0.026357,  # tap_accel_std_z
    0.023726,  # tap_accel_mean_freq
    0.020974,  # tap_accel_std_x
    0.019336,  # gait_accel_std_z
    0.019327,  # tap_accel_rms
    0.019190,  # Spectral_Centroid_mean
    0.017527,  # tap_accel_std_y
    0.017189   # gait_accel_rms_z
])

def softmax_temperature(x, T=1.0):
    e_x = np.exp((x - np.max(x)) / T)
    return e_x / e_x.sum()

n_qubits = len(feature_importances)
dev = qml.device("default.qubit", wires=n_qubits)

def aggregate_measurements(measurements):
    return np.sum(softmax_temperature(feature_importances, T=0.01) * measurements)

@qml.qnode(dev)
def quantum_kernel(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_kernel_function(x1, x2):
    measurements = quantum_kernel(x1, x2)
    return aggregate_measurements(measurements)

def kernel_matrix_with_logging(X1, X2):
    N = len(X1)
    M = len(X2)
    K = np.zeros((N, M))
    start_time = time.time()
    for i in range(N):
        for j in range(M):
            K[i, j] = quantum_kernel_function(X1[i], X2[j])
        if i % 10 == 1:
            elapsed_time = time.time() - start_time
            progress = (i * M) / (N * M)
            estimated_total_time = elapsed_time / progress if progress > 0 else 0
            estimated_remaining_time = estimated_total_time - elapsed_time
            print(f"Progress: {i}/{N} rows completed")
            print(f"Elapsed time: {elapsed_time:.2f} seconds")
            print(f"Estimated remaining time: {estimated_remaining_time:.2f} seconds")
    return K

**Train Model**

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

features_final = data_new.loc[:, 'age':]
labels_final = data_new['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(features_final.to_numpy(), labels_final.to_numpy(), test_size=0.15, random_state=122)

K_train = kernel_matrix(X_train, X_train)
K_test = kernel_matrix(X_test, X_train)

cld = SVC(kernel='precomputed')
cld.fit(K_train, y_train)

y_pred = cld.predict(K_test)
y_scores = cld.decision_function(K_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_scores)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC/AUC Score: {roc_auc:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.90
Precision: 0.90
Recall: 0.90
ROC/AUC Score: 0.97

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.91      0.87        11
           1       0.94      0.89      0.92        19

    accuracy                           0.90        30
   macro avg       0.89      0.90      0.89        30
weighted avg       0.90      0.90      0.90        30



In [14]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report

features_final = data_new.loc[:, 'age':]
labels_final = data_new['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(features_final.to_numpy(), labels_final.to_numpy(), test_size=0.15, random_state=122)

K_train = kernel_matrix_with_logging(X_train, X_train)
K_test = kernel_matrix_with_logging(X_test, X_train)

clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train)

y_pred = clf.predict(K_test)
y_scores = clf.decision_function(K_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_scores)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC/AUC Score: {roc_auc:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Progress: 1/164 rows completed
Elapsed time: 20.33 seconds
Estimated remaining time: 3313.64 seconds
Progress: 11/164 rows completed
Elapsed time: 102.57 seconds
Estimated remaining time: 1426.66 seconds
Progress: 21/164 rows completed
Elapsed time: 184.88 seconds
Estimated remaining time: 1258.94 seconds
Progress: 31/164 rows completed
Elapsed time: 269.80 seconds
Estimated remaining time: 1157.52 seconds
Progress: 41/164 rows completed
Elapsed time: 352.35 seconds
Estimated remaining time: 1057.06 seconds
Progress: 51/164 rows completed
Elapsed time: 433.89 seconds
Estimated remaining time: 961.37 seconds
Progress: 61/164 rows completed
Elapsed time: 547.77 seconds
Estimated remaining time: 924.93 seconds
Progress: 71/164 rows completed
Elapsed time: 651.25 seconds
Estimated remaining time: 853.05 seconds
Progress: 81/164 rows completed
Elapsed time: 737.24 seconds
Estimated remaining time: 755.44 seconds
Progress: 91/164 rows completed
Elapsed time: 825.57 seconds
Estimated remainin